# Natural Language Processing
After attending the workshops by Olaf Janssen [\[1\]](#References) on Natural Language Processing (NLP), <br>
it is time to start playing with state-of-the-art NLP pre-trained models, such as BERT (and derivatives such as ALBERT and RoBERTa), GPT-2 or XLNET. <br>
<br>
The assignment is to first pick a NLP task (such as Q&A, search engine, chatbot, classification, etc.). <br>
Then to choose a context wherein this task is being ran (such as a game, a service, smart applications, etc.). <br>
And, build it! <br>
The idea is that the student will be working with pre-trained models and that the focus is primarily on designing the right pipeline for the problem at hand. <br>
Endlessly fine-tuning the model is not a priority in this assignment; the goal is to demonstrate that the student understands how a NLP task is build up and can apply NLP techniques. <br>
The result must be an interactive prototype that tackles a specific NLP task.

# Table of Contents
[Introduction](#Introduction) <br>
[Preparation](#Preparation) <br>
[Conclusion](#Conclusion) <br>
[References](#References) <br>

# Introduction
This notebook

# Preparation
In this section useful libraries are imported which are used in most data science projects.

In [1]:
import os
import sys

# sets the path to the home directory of this repository so other modules can be imported. 
project_path = os.getcwd()
root_path = os.path.split(os.path.split(os.getcwd())[0])[0]
assert root_path.endswith("Fontys-ADS"), "The root path does not end with Fontys-ADS: " + root_path 
sys.path.insert(0, root_path)

import numpy as np
import tensorflow as tf


# set the seed for reproducible results.
np.random.seed(56)
tf.random.set_seed(56)

# optionally, set TensorFlow to use the GPU with all available memory.
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import transformers
print(f'transformers=={transformers.__version__}')
print(f'tensorflow=={tf.__version__}')

# Conclusion
conclusion

# References
\[1\] Janssen, O. (2020). NLP Introduction Overview. Olafjanssen.github.io. Retrieved 12 November 2020, from https://olafjanssen.github.io/nlp-workshop/.